In [1]:
import glob, os, ntpath, shutil, sys
from pathlib import Path
from ntpath import basename

import cv2
import h5py
import imageio
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from scipy.ndimage.interpolation import zoom
from skimage.transform import resize

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.python.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
tf.compat.v1.disable_eager_execution()

# Load CNN model

In [4]:
model_path = 'path/to/model.h5'
model = load_model(model_path, compile=False)

In [6]:
target_size = (331, 331)
layer_name = 'block5_conv3'
image_path = 'dataset/Pinus_koraiensis/BARK_KR.Pinus_koraiensis.177.jpg'
index = 21 # Use the ground truth index number for each class index

# Class Activation Mapping (CAM) Aggregation

In [7]:
y_c = model.output[0, index]
conv_output = model.get_layer(layer_name).output
grads = K.gradients(y_c, conv_output)[0]

first = K.exp(y_c)*grads
second = K.exp(y_c)*grads*grads
third = K.exp(y_c)*grads*grads*grads

gradient_function = K.function([model.input], [y_c, first,second,third, conv_output, grads])

In [ ]:
im = np.array(Image.open(image_path)) / 255.
h, w, _ = im.shape

w_prime = int(w * 0.4) if int(w * 0.4)%2==0 else int(w * 0.4) + 1
stride = int(w_prime / 2)

if w_prime < 331:
    w_prime = 330
    stride = int(w_prime / 2)

pad_w1, pad_w2 = stride, 2*stride - w%stride
pad_h1, pad_h2 = stride, 2*stride - h%stride
n_pads = ((pad_h1, pad_h2), (pad_w1, pad_w2), (0,0))

# Create empty image with the padded width & height
im_pad = np.pad(im, n_pads, 'constant', constant_values=0)
h_pad, w_pad, _ = im_pad.shape
R = np.zeros((h_pad, w_pad))

In [9]:
# Calculate number of iterations
N = int((h_pad - w_prime) / stride + 1)
M = int((w_pad - w_prime) / stride + 1)

for i in range(N):
    for j in range(M):
        i_, j_ = i * stride, j * stride
        crop_im = im_pad[i_:i_+w_prime, j_:j_+w_prime, :]
        crop_im = cv2.resize(crop_im, dsize=target_size, interpolation=cv2.INTER_NEAREST)
        img_tensor = np.expand_dims(crop_im, axis=0)

        y_c, conv_first_grad, conv_second_grad,conv_third_grad, conv_output, grads_val = gradient_function([img_tensor])
        global_sum = np.sum(conv_output[0].reshape((-1,conv_first_grad[0].shape[2])), axis=0)
        alpha_num = conv_second_grad[0]
        alpha_denom = conv_second_grad[0]*2.0 + conv_third_grad[0]*global_sum.reshape((1,1,conv_first_grad[0].shape[2]))
        alpha_denom = np.where(alpha_denom != 0.0, alpha_denom, np.ones(alpha_denom.shape))
        alphas = alpha_num/alpha_denom
        weights = np.maximum(conv_first_grad[0], 0.0)
        alpha_normalization_constant = np.sum(np.sum(alphas, axis=0),axis=0)
        alphas /= alpha_normalization_constant.reshape((1,1,conv_first_grad[0].shape[2]))
        deep_linearization_weights = np.sum((weights*alphas).reshape((-1,conv_first_grad[0].shape[2])),axis=0)
        
        grad_CAM_map = np.sum(deep_linearization_weights*conv_output[0], axis=2)
        cam = np.maximum(grad_CAM_map, 0)
        cam = zoom(cam,w_prime/cam.shape[0]) # Upscaling to w_prime
        cam = cam / np.max(cam) # scale 0 to 1.0
        cam[np.isnan(cam)] = 0
        R[i_:i_+w_prime, j_:j_+w_prime] += cam

R = R[stride:stride+h,stride:stride+w]
R = np.maximum(R, 0) # Retrieve only positive values
R = (R * 255 / 4).astype('uint8')

# Superimpose CAM Aggregation results on the original images

In [35]:
orig_img = cv2.imread(image_path)
CAM_img = cv2.applyColorMap(R, cv2.COLORMAP_JET)
alpha = 0.4
superimposed = alpha * CAM_img + (1 - alpha) * orig_img

In [36]:
cv2.imwrite('sample.jpg', superimposed)

True

![](sample.jpg)